In [ ]:
import json
import requests

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error

sns.set_palette('Set2')
%matplotlib inline

In [ ]:
endpoint = 'https://min-api.cryptocompare.com/data/histoday'
res = requests.get(endpoint + '?fsym=BTC&tsym=USD&limit=2000')
hist = pd.DataFrame(json.loads(res.content)['Data'])
hist = hist.set_index('time')
hist.index = pd.to_datetime(hist.index, unit='s')



In [ ]:
hist.head()

In [ ]:

target_col = 'close'
def train_test_split(df, test_size=0.1):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

train, test = train_test_split(hist, test_size=0.1)

In [ ]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(16, 9))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('price [USD]', fontsize=14)
    ax.set_title(title, fontsize=18)
    ax.legend(loc='best', fontsize=18)

line_plot(train[target_col], test[target_col], 'training', 'test', title='BTC')

In [ ]:
def normalise_zero_base(df):
    """ Normalise dataframe column-wise to reflect changes with respect to first entry. """
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    """ Normalise dataframe column-wise min/max. """
    return (df - df.min()) / (data.max() - df.min())

def extract_window_data(df, window_len=10, zero_base=True):
    """ Convert dataframe to overlapping sequences/windows of len `window_data`.
    
        :param window_len: Size of window
        :param zero_base: If True, the data in each window is normalised to reflect changes
            with respect to the first entry in the window (which is then always 0)
    """
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)



def prepare_data(df, target_col, window_len=10, zero_base=True, test_size=0.2):
    """ Prepare data for LSTM. """
    # train test split
    train_data, test_data = train_test_split(df, test_size=test_size)
    
    # extract window data
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    
    # extract targets
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test


def build_lstm_model(input_data, output_size, neurons=20, activ_func='linear',
                     dropout=0.25, loss='mae', optimizer='adam'):
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model




In [ ]:
np.random.seed(42)

# data params
window_len = 7
test_size = 0.1
zero_base = True

# model params
lstm_neurons = 20
epochs = 50
batch_size = 4
loss = 'mae'
dropout = 0.25
optimizer = 'adam'

In [ ]:
train, test, X_train, X_test, y_train, y_test = prepare_data(
    hist, target_col, window_len=window_len, zero_base=zero_base, test_size=test_size)

model = build_lstm_model(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
history = model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)



In [ ]:
targets = test[target_col][window_len:]
preds = model.predict(X_test).squeeze()

mean_absolute_error(preds, y_test)

preds = test[target_col].values[:-window_len] * (preds + 1)
preds = pd.Series(index=targets.index, data=preds)

line_plot(targets, preds, 'actual', 'prediction', lw=3)

In [ ]:
n_points = 30

line_plot(targets[-n_points:], preds[-n_points:], 'actual', 'prediction', lw=3)

In [ ]:
actual_returns = targets.pct_change()[1:]
predicted_returns = preds.pct_change()[1:]

def dual_line_plot(line1, line2, line3, line4, label1=None, label2=None, title='', lw=2):
    import matplotlib.dates as mdates
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(21, 9))
    ax1.plot(line1, label=label1, linewidth=lw)
    ax1.plot(line2, label=label2, linewidth=lw)
    ax2.plot(line3, label=label1, linewidth=lw)
    ax2.plot(line4, label=label2, linewidth=lw)
    ax2.set_xticks(ax1.get_xticks())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax1.set_ylabel('daily returns', fontsize=14)
    ax2.legend(loc='best', fontsize=18);

In [ ]:
dual_line_plot(actual_returns[-n_points:],
          predicted_returns[-n_points:],
          actual_returns[-n_points:][:-1],
          predicted_returns[-n_points:].shift(-1),
          'actual returns', 'predicted returns', lw=3)

In [ ]:
line_plot(actual_returns[-n_points:][:-1], predicted_returns[-n_points:].shift(-1),
           'actual returns', 'predicted returns', lw=3)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))

# actual correlation
corr = np.corrcoef(actual_returns, predicted_returns)[0][1]
ax1.scatter(actual_returns, predicted_returns, color='k', marker='o', alpha=0.5, s=100)
ax1.set_title('r = {:.2f}'.format(corr), fontsize=18)

# shifted correlation
shifted_actual = actual_returns[:-1]
shifted_predicted = predicted_returns.shift(-1).dropna()
corr = np.corrcoef(shifted_actual, shifted_predicted)[0][1]
ax2.scatter(shifted_actual, shifted_predicted, color='k', marker='o', alpha=0.5, s=100)
ax2.set_title('r = {:.2f}'.format(corr), fontsize=18);

In [17]:
import requests
import pandas as pd
import xlsxwriter

# apiKey = "3d962fc56013560002625148bb9d05b2713e2415f63cf9bf25ccab7612b0de6f" #Karthikeyan
apiKey = "fc90914ed266b5c6c5277e07be3878269cfcc6f4c22be38c4cb8df6b9de3752a" #deweshcrypto

url = "https://min-api.cryptocompare.com/data/histoday"
symbols = ["BTC", "BCH", "DASH", "ETH", "ETC", "LTC", "REP"]

headers = {
    "authorization": "Apikey " + apiKey
}

currency = "USD"

for sym in symbols: 
    payload = {
        "api_key": apiKey,
        "fsym": sym,
        "tsym": currency,
        "limit": 500
    }

    # result = requests.get(url, params=payload).json()
    result = requests.get(url, headers=headers, params=payload).json()
    s_name = sym + '_in_' + currency
    f_name = s_name + '.xlsx'
    
    df = pd.DataFrame(result['Data'])
    df = df[['time', 'close', 'high', 'low', 'open', 'volumefrom', 'volumeto']]
    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='s')
    
    writer = pd.ExcelWriter(f_name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name=s_name)
    writer.save()
    
    print("=========================================")
    print(s_name)
    print("=========================================")
    print(df.head())

BTC_in_USD
              close     high      low     open  volumefrom      volumeto
time                                                                    
2018-02-10  8569.29  9081.49  8176.25  8696.83   155616.78  1.348923e+09
2018-02-11  8084.61  8573.35  7862.31  8569.32   123293.84  1.013772e+09
2018-02-12  8911.27  8997.34  8084.41  8084.61   124923.98  1.085922e+09
2018-02-13  8544.69  8955.15  8379.35  8911.17    98632.88  8.533204e+08
2018-02-14  9485.64  9508.22  8542.98  8544.69   133702.54  1.220863e+09
BCH_in_USD
              close     high      low     open  volumefrom     volumeto
time                                                                   
2018-02-10  1240.62  1381.63  1198.68  1299.97    73317.51  93963908.26
2018-02-11  1210.99  1298.30  1188.47  1240.54    54895.10  67332282.15
2018-02-12  1282.63  1293.05  1208.89  1211.03    31886.53  40345803.75
2018-02-13  1220.89  1285.27  1200.31  1282.05    36215.01  44591425.59
2018-02-14  1353.18  1376.41  1220.

In [8]:
def get_data(coin, currency, date):
    """ Query the API for 2000 days historical price data starting from "date". """
    
    url = "https://min-api.cryptocompare.com/data/histoday?fsym="+coin+"&tsym="+currency+"&limit=2000&api_key="+apiKey+"&toTs={}".format(date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [16]:
import requests
import pandas as pd
import xlsxwriter

# apiKey = "3d962fc56013560002625148bb9d05b2713e2415f63cf9bf25ccab7612b0de6f" #Karthikeyan
apiKey = "fc90914ed266b5c6c5277e07be3878269cfcc6f4c22be38c4cb8df6b9de3752a" #deweshcrypto

symbols = ["BTC", "BCH", "DASH", "ETH", "ETC", "LTC", "REP", "XRH"]
currency = "USD"
from_date = 1514764800

print("1314764800".format(from_date))

for coin in symbols: 
    result = get_data(coin, currency, from_date)
    s_name = sym + '_in_' + currency
    f_name = s_name + '.xlsx'
    
    df = pd.DataFrame(result['Data'])
    df = df[['time', 'close', 'high', 'low', 'open', 'volumefrom', 'volumeto']]
    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='s')
    
    writer = pd.ExcelWriter(f_name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name=s_name)
    writer.save()
    
    print("=========================================")
    print(s_name)
    print("=========================================")
    print(df.head())

1314764800
XRH_in_USD
            close   high    low   open  volumefrom   volumeto
time                                                         
2012-07-11  7.150  7.257  7.018  7.200    41891.36  297644.88
2012-07-12  7.764  7.900  7.100  7.150    64616.60  471834.37
2012-07-13  7.665  7.816  7.420  7.764    65465.74  499441.51
2012-07-14  7.542  7.665  7.516  7.665    16007.10  121635.99
2012-07-15  7.621  7.700  7.450  7.542    26978.30  204616.50
XRH_in_USD
            close  high  low  open  volumefrom  volumeto
time                                                    
2012-07-11    0.0   0.0  0.0   0.0         0.0       0.0
2012-07-12    0.0   0.0  0.0   0.0         0.0       0.0
2012-07-13    0.0   0.0  0.0   0.0         0.0       0.0
2012-07-14    0.0   0.0  0.0   0.0         0.0       0.0
2012-07-15    0.0   0.0  0.0   0.0         0.0       0.0
XRH_in_USD
            close  high  low  open  volumefrom  volumeto
time                                                    
2012-07-1

KeyError: "None of [Index(['time', 'close', 'high', 'low', 'open', 'volumefrom', 'volumeto'], dtype='object')] are in the [columns]"